In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import get_description, get_html_content, get_links
from urllib.parse import urljoin, urlparse, parse_qs, urlencode


In [3]:
def get_job_links(html_content):
    links = html_content.select('a.title')
    jobs_listing_links = [a.get("href") for a in links if a.get("href")]
    return jobs_listing_links

#return pagination links
def get_pages(html_content,base_url):
    pagination_links = html_content.select("div.styles_pages__v1rAK a[href]")
    parsed_url = urlparse(base_url)
    query_params = parse_qs(parsed_url.query)
    query_string = urlencode(query_params, doseq=True)
    base_domain = "https://www.naukri.com"
    page_urls = []
    for a_tag in pagination_links:
        href = a_tag.get("href")
        full_path = urljoin(base_domain, href)
        full_url_with_query = f"{full_path}?{query_string}"
        page_urls.append(full_url_with_query)
    return page_urls

In [5]:
base_url = 'https://www.naukri.com/react-dot-js-python-azure-devops-jobs-in-bengaluru?experience=3&cityTypeGid=97&ctcFilter=15to25&ctcFilter=25to50&ctcFilter=50to75&ctcFilter=75to100&ctcFilter=100to500&jobAge=1'
html_content = get_html_content.get_url_data(base_url)
pages = get_links.get_pages(html_content,base_url) #this will return the pages
job_listing = []
job_listing_page1 = get_links.get_job_links(html_content) #this will contains links of all jobs
job_listing.extend(job_listing_page1)

if len(pages)>1:  #getting the job listings of other pages
    for url in pages:
        html_content = get_html_content.get_url_data(url)
        job_listing_page1 = get_links.get_job_links(html_content) #this will contains links of all jobs
        job_listing.extend(job_listing_page1)
job_description = []
print("totol number of jobs", len(job_listing))
# for url in job_listing[:3]:
#     details = {}
#     details['url'] = url
#     html_content = get_html_content.get_url_data(url)
#     description_json = get_description.extract_flexible_job_description(html_content)
#     details['desc'] = description_json
#     job_description.append(details)
# print(job_description)

totol number of jobs 104


In [37]:
def get_AI_skill_score(details):
    import google.generativeai as genai
    import os

    from dotenv import load_dotenv
    load_dotenv()

    # Configure the Gemini model
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")

    # Your job description and resume texts
    job_description = details

    resume = """
Proficient software engineer with 3 years of experience in developing Python Systems or Services. I am experienced in enhancing applications and managing full-cycle deployments using Azure DevOps. Currently exploring AI-based solutions and automation tools like n8n, OpenAI to drive continuous improvement and efficiency.

PROFESSIONAL EXPERIENCE
ARM Embedded Technologies | Sept 2024 - Present  | Support Specialist: Digital Platform Full Stack
- Developed a service in Python to update and validate user details across multiple systems.
- Built Python scripts for bulk user detail updates and data integrity checks.
- Created many alerts in Azure using KQL.
- Automated monitoring reports using Azure Automation with Python.
- Deployed application code to production and pre-production environments using Azure DevOps.
- Implemented a Power BI dashboard for application performance.
- Worked on UI enhancements and feature development using React JS.

Tata Consultancy Services | Systems Engineer | Oct 2021 - Sept 2024
- Created file-processing and database services using Python based on business needs.
- Scheduled cron jobs in Azure DevOps using Python.
- Implemented new features in React JS based on business needs.
- Built a dashboard for monitoring application performance.
- Managed production deployments and release cycles with Azure DevOps.
- Analyzed reports, identified root causes, and ensured system stability.
- Used KQL extensively to diagnose system issues.
- Awarded Star of the Quarter (Oct 2022).
- Top 10 participant in Tata Neu app improvement ideathon.
"""

    # Prompt template
    prompt = f"""
    You are an expert career assistant.

    Compare the following job description and candidate resume. 

    1. Evaluate how well the candidate's skills and experience match the job requirements.
    2. Provide a match score between 0 and 1 (where 1 is a perfect match).
    3. Give a brief reason for your assessment.

    Return the result in the following JSON format:
    {{
    "score": float (0 to 1),
    "fit": "Good" or "Bad",
    "reason": "Brief explanation"
    }}

    Job Description:
    {job_description}

    Resume:
    {resume}
    """

    # Generate the response
    response = model.generate_content(prompt)
    return response

    # Print the result
    


In [6]:
# print("totol number of jobs", len(job_listing))
jobs_to_apply =[]
for url in job_listing[:1]:
    details = {}
    details['url'] = url
    html_content = get_html_content.get_url_data(url)
    description_json = get_description.extract_flexible_job_description(html_content)
    details['desc'] = description_json
    # match = get_skill_score(details['desc'])
    # if match.get('score') >0.8:
    # match['url'] = url
    # jobs_to_apply.append(match)

    # job_description.append(details)
# print(jobs_to_apply)

{'General Description': 'Software Development Engineer\nExperience:\n2 - 6 Years Exp\nSalary :\nCompetitive\nPreferred Notice Period\n: Within 15 Days\nOpportunity Type:\nOnsite (Bengaluru)\nPlacement Type:\nPermanent\n(*Note: This is a requirement for one of Uplers\' Clients)\nMust have skills required :\nPython, react, FastAPI\nInfibeam (One of Uplers\' Clients) is Looking for:\nSoftware Development Engineer\nwho is passionate about their work, eager to learn and grow, and who is committed to delivering exceptional results. If you are a team player, with a positive attitude and a desire to make a difference, then we want to hear from you.\nRole Overview Description\nBackend/Fullstack Engineer - Phronetic AI (a division of Infibeam Avenues Limited)\nWho We Are\nPhronetic, the artificial intelligence division of Infibeam Avenues Limited, is building the foundational platform for the "agentic economy," moving beyond simple chatbots to create an ecosystem for autonomous AI agents. Powere

In [ ]:
import re
# response = get_skill_score(details['desc'])


d:\AI\starter_pack\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
import re
import json

# Step 1: Get the text content safely from the response object
response_text = response.candidates[0].content.parts[0].text

# Step 2: Remove ```json ... ``` wrappers if present
match = re.search(r"```json(.*?)```", response_text, re.DOTALL)
if match:
    clean_json = match.group(1).strip()
else:
    clean_json = response_text.strip()

# Step 3: Load the JSON into a Python dict
result = json.loads(clean_json)

# Step 4: Print or return the result
print(result)


{'score': 0.6, 'fit': 'Good', 'reason': "The candidate demonstrates experience with Python, Azure DevOps, and monitoring (though not specifically with the listed tools like AppDynamics).  Their experience aligns with several key aspects of the job description, including deployment, automation, and using Azure for development and monitoring. However, the candidate lacks explicit experience with Java, Kubernetes, Prometheus, Node.JS, and  doesn't mention experience with other cloud providers like AWS. The experience with KQL is relevant, but the job description specifically calls out several monitoring tools.  The overall experience level is sufficient, but the skillset needs some further development to perfectly match the job requirements."}


In [ ]:
# def get_skill_score(html_content):
#     key_skills = [
#     "Python",
#     "React JS",
#     "JavaScript",
#     "HTML",
#     "CSS",
#     "Flask",
#     "MySQL",
#     "PL/SQL",
#     "KQL",
#     "Azure DevOps",
#     "Azure Automation",
#     "Power BI",
#     "n8n",
#     "OpenAI",
#     "CI/CD",
#     "Agile",
#     "System Monitoring",
#     "Troubleshooting",
#     "Data Structures and Algorithms",
#     "Unit Testing"
# ]

#     job_cards = html_content.select('div.cust-job-tuple')
#     # print(job_cards)
#     job_data = []

#     for card in job_cards:
#         # Extract URL
#         url_tag = card.select_one('div.row1 h2 a.title')
#         url = url_tag['href'].strip() if url_tag and url_tag.has_attr('href') else None

#         # Extract tags/skills
#         tags = [li.get_text(strip=True).lower() for li in card.select('div.row5 ul.tags-gt li')]
#         # print(url)
#         if url!=None:
#             job_data.append({
#                 "url": url,
#                 "skills": tags
#             })

# # Output
#     for job in job_data:
#         print(f"URL: {job['url']}")
#         print(f"Skills: {job['skills']}")
#         print("-" * 40)

In [ ]:
# base_url = 'https://www.naukri.com/react-dot-js-python-azure-devops-jobs-in-bengaluru?experience=3&cityTypeGid=97&ctcFilter=15to25&ctcFilter=25to50&ctcFilter=50to75&ctcFilter=75to100&ctcFilter=100to500&jobAge=1'
# html_content = get_html_content.get_url_data(base_url)
get_skill_score(html_content)

URL: https://www.naukri.com/job-listings-sre-devops-n-infosys-hyderabad-pune-bengaluru-3-to-8-years-100625021306
Skills: ['aws devops', 'sre', 'site reliability engineering', 'devops', 'azure devops', 'azure cloud', 'node.js', 'dynatrace']
----------------------------------------
URL: https://www.naukri.com/job-listings-fullstack-engineer-uplers-bengaluru-2-to-6-years-030725026063
Skills: ['python', 'backend', 'django', 'fast api', 'full stack', 'react.js', 'python development', 'aws']
----------------------------------------
URL: https://www.naukri.com/job-listings-full-stack-developer-innointel-global-bengaluru-3-to-6-years-070625009447
Skills: ['artificial intelligence', 'react native', 'rest api', 'cloudfront', 'python', 'websocket', 'aws cloud', 'amplify']
----------------------------------------
URL: https://www.naukri.com/job-listings-devops-engineer-aws-azure-sairam-global-technologies-hyderabad-pune-bengaluru-3-to-8-years-030725012562
Skills: ['terraform', 'ci cd pipeline', 'a

<html lang="en"><head><meta charset="utf-8"/><link href="https://www.googletagmanager.com" rel="dns-prefetch"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/4d9334c997853846.css" rel="stylesheet"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/3c9d38be1ad0b202.css" rel="stylesheet"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/ef49947185a39b61.css" rel="stylesheet"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/97d28480c758bae1.css" rel="stylesheet"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/bdd70167d05b00ce.css" rel="stylesheet"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/cd7d7f5f7e6df4d5.css" rel="stylesheet"/><link data-precedence="next" href="https://static.naukimg.com/s/9/121/_next/static/css/dadd1e287287e304.css" rel="stylesheet"/>

In [ ]:
import requests
import json

# Ollama API endpoint (default when running Ollama locally)
OLLAMA_URL = 'http://localhost:11434/api/generate'

# Your dynamic resume (replace with your actual text or load from a file)
resume = """
Proficient software engineer with 3 years of experience in developing Python Systems or Services. Experienced in Azure DevOps, automation, AI tools (n8n, OpenAI), React, KQL.
PROFESSIONAL EXPERIENCE
ARM Embedded Technologies | Sept 2024 - Present  | Support Specialist: Digital Platform Full Stack
- Developed a service in Python to update and validate user details across multiple systems.
- Built Python scripts for bulk user detail updates and data integrity checks.
- Created many alerts in Azure using KQL.
- Automated monitoring reports using Azure Automation with Python.
- Deployed application code to production and pre-production environments using Azure DevOps.
- Implemented a Power BI dashboard for application performance.
- Worked on UI enhancements and feature development using React JS.

Tata Consultancy Services | Systems Engineer | Oct 2021 - Sept 2024
- Created file-processing and database services using Python based on business needs.
- Scheduled cron jobs in Azure DevOps using Python.
- Implemented new features in React JS based on business needs.
- Built a dashboard for monitoring application performance.
- Managed production deployments and release cycles with Azure DevOps.
- Analyzed reports, identified root causes, and ensured system stability.
- Used KQL extensively to diagnose system issues.
- Awarded Star of the Quarter (Oct 2022).
- Top 10 participant in Tata Neu app improvement ideathon.
"""

# Example dynamic job description (replace or loop through your scraped jobs)
job_description = """
{'General Description': 'Job description\nHiring for site reability engineer - AWS /Azure Devops with experience range 3 + years.\nMandatory Skills:\nJava, Kubernetes, AWS/Azure, DevOps/DevSecOps, Monitoring Tools - App Dynamics/ Dynatrace/New Relic, Build and Release, Prometheus, Python, Node.JS-site reability engineer\nEducation: BE/B.Tech/MCA/M.Tech/MSc./MSts'}
"""

# Prompt template with clear JSON instruction
prompt = f"""
You are an expert career assistant. Given a job description and a candidate's resume, assess how well the candidate matches the job.

Return only a JSON in this format:
{{
  "score": float between 0 and 1,
  "fit": "Good" or "Bad",
  "reason": "Brief reason"
}}

Resume:
{resume}

Job:
{job_description}
"""

# Request payload
data = {
    "model": "phi3:3.8b",
    "prompt": prompt,
    "stream": False
}

# Call Ollama locally
response = requests.post(OLLAMA_URL, json=data)

# Parse response
result = response.json().get('response', '')

print("LLM Response:\n", result)

# Optional: Parse JSON from response if needed
try:
    json_start = result.find('{')
    json_data = json.loads(result[json_start:])
    print("\nParsed Result:", json_data)
except Exception as e:
    print("\nFailed to parse JSON:", e)


In [ ]:
from google import genai

def get_match(resume: str, job_desc: str):
    prompt = build_prompt(resume, job_desc)
    response = genai.Client().models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=[{"text": prompt}],
        # Optional: disable deep thinking to save tokens
        # config=genai.types.GenerateContentConfig(thinking_config=genai.types.ThinkingConfig(thinking_budget=0)),
    )
    return response.text


In [ ]:
import json
import pandas as pd

# Example resume text
resume_text = """..."""  # Replace with your rolled-up resume

df['score'] = -1.0
df['match'] = ''
df['reason'] = ''

for idx, row in df.iterrows():
    res = get_match(resume_text, row['desc'])
    try:
        # Safely extract JSON substring
        json_part = res[res.find('{'):res.rfind('}')+1]
        data = json.loads(json_part)
        df.at[idx, 'score'] = data.get('score', -1)
        df.at[idx, 'match'] = data.get('fit', '')
        df.at[idx, 'reason'] = data.get('reason', '')
    except Exception as e:
        print(f"Error at row {idx}: {e}")
